# check

In [69]:
from sympy import *
import numpy as np
import math
import random
import tools as my
import time

def SquarefreeDecomposition(f):
    if content(f) != 1:
        raise ValueError('cont(f) != 1')
    s = gcd(f, diff(f, f.gen))
    if s == 1:
        return [f]
    t = []; u = []; v = []; F = []
    t.append(quo(f, s)); u.append(quo(diff(f, f.gen), s))
    v.append(u[0] - diff(t[0]))
    i = 1
    while t[i-1] != 1:
        F.append(gcd(t[i-1], v[i-1]))
        t.append(quo(t[i-1], F[i-1]))
        u.append(quo(v[i-1], F[i-1]))
        v.append(u[i] - diff(t[i], f.gen))
        i = i + 1
    return F

def Berlekamp_Matrix(f):
    # Berlekamp行列
    
    if LC(f) != 1:
        raise ValueError('f(x) is not monic.')
    if gcd(f, diff(f, f.gen, domain=f.domain)) != 1:
        raise ValueError('f(x) is not square-free.')

    x = f.gen
    p = f.domain.characteristic()
    m = degree(f)
    g0 = 1
    B = np.zeros((m, m), dtype=Integer); B[0][0] = g0
    g1 = rem(x**p, f, domain=GF(p))
    if g1 == g1.zero:
        deg_g1 = 0
    else:
        deg_g1 = degree(g1)
    for i in range(1, deg_g1+1):
        B[1][i-1] = list(reversed(g1.all_coeffs()))[i]
    g = g1
    for i in range(2, m):
        g = rem(g*g1, f, domain=GF(p))
        for j in range(1, degree(g)+2):
            B[i][j-1] = g.all_coeffs()[-j]
    return B

def f_ReducingPolynomial(f, B):
    # f-簡約多項式（Berlekampアルゴリズムでの分離多項式）
    
    x = f.gen
    p = f.domain.characteristic()
    m = np.shape(B)[0]
    B = B - np.identity(m, dtype=Integer)
    for i in range(m-1):
        if sum(B[i][i:] == 0) == m-i:
            continue
        j = i
        while B[i][i] == 0:
            j = j + 1
            if B[i][j] != 0:
                B[:, i], B[:, j] = B[:, j], B[:, i]    
        recip = invert(B[i][i], p)
        B[:, i] = (B[:, i]*recip) % p
        for j in range(m):
            if j == i:
                continue
            B[:, j] = (B[:, j] - B[i][j]*B[:, i]) % p
    B = np.identity(m, dtype=Integer) - B
    
    _B = np.zeros((1, m), dtype=Integer)
    for i in range(m):
        if sum(B[i] == 0) == m:
            continue
        _B = np.append(_B, [B[i]], axis=0)
    _B = _B[1:]
    f_reducing_polynomials = []
    
    k = np.shape(_B)[0]
    for i in range(k):
        _f = Poly(0, x, domain=GF(p))
        for j in range(m):
            coeff = _B[i][j]
            _f = _f + Poly(coeff*x**j, x, domain=GF(p))  
        f_reducing_polynomials.append(_f)
    return tuple(f_reducing_polynomials)

def _BerlekampAlgorithm(f, output=True):
    # Berlekampアルゴリズム（有限体上の因数分解）
    
    if LC(f) != 1:
        raise ValueError('f(x) is not monic.')
    if gcd(f, diff(f, f.gen)) != 1:
        raise ValueError('f(x) is not square-free.')
        
    B = Berlekamp_Matrix(f)
    f_reducing_polynomials = f_ReducingPolynomial(f, B)
    
    if output:
        print('Berlekamp Matrix: ' + '\n' + str(B) + '\n')
        print('f reducing polynomials:' + '\n' + str(f_reducing_polynomials) + '\n')

        print('--------------------_Berlekamp Algorithm--------------------')
    
    # 自明なf-簡約多項式はg1へ
    g1 = np.array([], dtype=Integer); g_list = np.array([], dtype=Integer)
    for g in f_reducing_polynomials:
        if degree(g) == 0:
            g1 = np.append(g1, g)
        else:
            g_list = np.append(g_list, g)
    if output:      
        print('g1 :' + str(g1)); print('g_list :' + str(g_list))
    
    #if len(f_reducing_polynomials) == 1:
    if len(g_list) == 1:
        return f
    else:
        F = np.array([f])
    alpha = range(f.domain.characteristic())
    
    # _F ... F', F_ ... F_tilde
    for i in range(len(g_list)):
        _F = np.array([], dtype=Integer)
        for _f in F:
            F_ = np.array([], dtype=Integer)
            for a in alpha:
                gcd_fg = gcd(_f, g_list[i] - a, domain=_f.domain)
                if degree(gcd_fg) != 0:
                    F_ = np.append(F_, gcd_fg)
            h = Poly(1, x, domain=_f.domain)
            for F_elements in F_:
                h = h * F_elements
            _f = quo(_f, h, domain=_f.domain)
            if _f != f.one:
                F_ = np.append(F_, _f)
            _F = np.append(_F, F_)
            print('_F:', _F)
        if len(_F) == len(f_reducing_polynomials):
            if output:
                print('rF', _F)
                print('_rF', tuple(_F))
                print('------------------------------------------------------------')
            return tuple(_F)
        F = _F

def _Zassenhaus(f):
    if primitive(f)[0] != 1:
        raise ValueError('f(x) is not primitive.')
    if gcd(f, diff(f, f.gen, domain=f.domain)) != 1:
        raise ValueError('f(x) is not square-free.')
        
    m = degree(f)
    max_f = f.max_norm()
    B_ = sqrt(m+1) * 2**m * max_f
    B = B_ * abs(LC(f))
    x = f.gen
    
    i = 10
    while True:
        i += 3
        p = prime(i)
        if (gcd(f, diff(f, x, domain=GF(p)), domain=GF(p)) == 1) and (bool(LC(f) % p)):
            break
    fp = Poly(f * invert(LC(f), p), x, domain=GF(p))
    print('fp:', fp)
    g_list = [_BerlekampAlgorithm(fp)]
    d = math.ceil(log(2*B+1, p)) - 1
    h_list = my.HenselLifting_MultiFactors(p, f, g_list, d, output=False)
    s = 1
    F = []; H = h_list; _f = f
    while 2*s <= len(H):
        iter_list = itertools.combinations(H, s)
        for S in iter_list:
            h_setminus_S = list(set(H) - set(S))
            f_s = Poly(LC(_f) * np.prod(S), x, domain=GF(p**(d+1))).as_poly(domain=ZZ)
            f_h_setminus_S = Poly(LC(_f) * np.prod(h_setminus_S), x, domain=GF(p**(d+1))).as_poly(domain=ZZ)
            if f_s.l1_norm() * f_h_setminus_S.l1_norm() <= B:
                H = h_setminus_S
                F.append(primitive(f_s)[1])
                _f = primitive(f_h_setminus_S)[1]
                break
        else:
            s = s + 1
    return np.append(F, _f)  

def _Zassenhaus_Algorithm(f):
    cont_f, pp_f = primitive(f)
    sf_f_list = SquarefreeDecomposition(pp_f)
    factors_list = []
    i = 1
    for sf_f in sf_f_list:
        print(sf_f)
        result = list(_Zassenhaus(sf_f))
        for _ in range(i):
            factors_list.append(result)
        i += 1
    factors_list[0] *= cont_f
    return factors_list

x = Symbol('x')
f = ((x+1)**2*(2*x-3)**3*(x**2+x+9)).as_poly(domain=ZZ)

In [70]:
%%time
_Zassenhaus_Algorithm(f)

Poly(x**2 + x + 9, x, domain='ZZ')
fp: Poly(x**2 + x + 9, x, modulus=41)
Berlekamp Matrix: 
[[1 0]
 [-1 0]]

f reducing polynomials:
(Poly(1, x, modulus=41), Poly(2*x + 1, x, modulus=41))

--------------------_Berlekamp Algorithm--------------------
g1 :[Poly(1, x, modulus=41)]
g_list :[Poly(2*x + 1, x, modulus=41)]


AttributeError: module 'tools' has no attribute 'HenselLifting_MultiFactors'

# memo

In [ ]:
# Comparisons of factorization over GF

In [101]:
x = Symbol('x')
p = prime(200)
p = 41
deg = 2
f = random_poly(x, deg, -p//2, p//2, domain=GF(p), polys=True)
f = monic(f)
print(f)

Poly(x**2 + 20*x + 11, x, modulus=41)


In [102]:
%%time
factor(f, domain=f.domain)

Wall time: 4.02 ms


x**2 + 20*x + 11

In [103]:
%%time
my._BerlekampAlgorithm(f)

Wall time: 55.6 ms


In [104]:
%%time
my.BerlekampAlgorithm(f)

Wall time: 23.8 ms


In [105]:
%%time
my.Cantor_Zassenhaus(f)

Wall time: 8.82 ms


[Poly(x**2 + 20*x + 11, x, modulus=41)]

In [76]:
%%time
if __name__ == '__main__':
    '''
    動作確認
    
    '''
    x = Symbol('x')
    p = prime(10); print(p)
    deg = 15
    factor_num = 3
    factor_deg = deg//factor_num
    f = Poly(1, x, domain=GF(p))
    i = 0
    while True:
        i += 1; print('\r%d' %i, end='')
        for _ in range(factor_num):
            _f = random_poly(x, factor_deg, -p//2, p//2, domain=GF(p), polys=True)
            f *= _f
        if gcd(f, diff(f, f.gen)) == 1:
            break
        else:
            f = Poly(1, x, domain=GF(p))
    f = monic(f)
    print(f)

29
1Poly(x**15 - 7*x**14 - 8*x**13 - 4*x**11 - 11*x**10 - 11*x**9 - 6*x**8 - 7*x**7 - 10*x**6 - 8*x**5 - 13*x**4 + 4*x**3 - 10*x**2 + 9*x + 9, x, modulus=29)
Wall time: 35.5 ms


In [88]:
g = Poly(x**2 + x + 9, x, modulus=41)

# main

In [ ]:
from sympy import *
import numpy as np
import math
import random
import tools as my
import time
import matplotlib.pyplot as plt
import pickle

In [ ]:
iter_num = 5
prime_sup = 50
print(prime(prime_sup))
deg = 15

Poly_list = []

x = Symbol('x')

dir_name = '../Soturonn/images/'

In [ ]:
%%time
# cache 用のprimeの呼び出し(about 70 sec)
_ = [my.my_prime(i) for i in range(1, prime_sup+1)]

In [ ]:
%%time
# 多項式の生成・保存
for i in range(1, prime_sup):
    p = my.my_prime(i)
    _list = []
    for _ in range(iter_num):
        while True:
            f = random_poly(x, deg, -p//2, p//2, domain=GF(p), polys=True)
            if gcd(f, diff(f, x, domain=f.domain), domain=f.domain) == 1:
                break
        f = monic(f)
        _list.append(f)
    Poly_list.append(_list)
        
#with open('Poly_lists.pickle', 'wb') as file:
 #   pickle.dump(Poly_list, file)

In [ ]:
%%time
func_list = [my._BerlekampAlgorithm, my.BerlekampAlgorithm, my.Cantor_Zassenhaus]
time_list = []
SP_time = []

for polys in Poly_list:
    t1 = time.process_time()
    for f in polys:
        factor(f, domain=f.domain)
    t2 = time.process_time()
    SP_time.append((t2 - t1)/iter_num)
time_list.append(SP_time)

for func in func_list:
    _time = []
    for polys in Poly_list:
        t1 = time.process_time()
        for f in polys:
            func(f)
        t2 = time.process_time()
        _time.append((t2 - t1)/iter_num)
    time_list.append(_time)
    
#with open('time_list.pickle', 'wb') as file:
 #   pickle.dump(time_list, file)

In [ ]:
%%time
fig, ax = plt.subplots()
x_list = [my.my_prime(i) for i in range(1, prime_sup)]
ax.plot(x_list, time_list[0], label='SymPy original')
ax.plot(x_list, time_list[1], label='Berlekamp algorithm')
ax.plot(x_list, time_list[2], label='Berlekamp algorithm(optimize)')
ax.plot(x_list, time_list[3], label='Cantor Zassenhaus algorithm')
ax.legend()
#ax.set_yscale('log')
ax.set_xlabel('"i"th prime')
ax.set_ylabel('time[sec]')

fig.tight_layout()
fig.show()
#plt.savefig(dir_name + 'GF_experiment.eps')
plt.savefig('sample.eps')